In [86]:
import os
import sys
import shutil
import csv
import numpy as np
from PIL import Image
from tabulate import tabulate

#i percorsi dovrebbero essere modificati nel momento in cui lo script viene eseguito su un altro pc, 
#stesso discorso per lo script addfile.py
#mi scuso per l'inconveniente

def dirchecker(directory): #questa funzione esegue un check della directory "files" e controlla se le cartelle dedicate ai file sono presenti o devono essere create
    os.chdir("C:\\Users\\manci\\Desktop\\FileOrganizer\\files") #il metodo chdir ci posiziona nella cartella files
    audcheck = "False" #i seguenti sono i 3 checker per controllare la presenza delle cartelle 
    imgcheck = "False"
    doccheck = "False"
    for items in directory:  #questo ciclo controlla ogni elemento nella directory files, nel momento in cui trova una cartella già creata con lo stesso nome delle cartelle  da creare allora setta la variabili di controllo adibita su true
        if items == "audio":
            audcheck = "True"
        elif items == "img":
            imgcheck = "True"
        elif items == "documents":
            doccheck = "True"
    if audcheck == "False": #nel momento in cui i check rimangono settati su false le cartelle vengono create 
        os.mkdir("audio")
    if imgcheck == "False":
        os.mkdir("img")
    if doccheck == "False":
        os.mkdir("documents")
    
def dirmover(directory):
    os.chdir("C:\\Users\\manci\\Desktop\\FileOrganizer\\files")
    with open('recap.csv', mode='a') as recap_file: #apertura del file csv
        recap_writer = csv.writer(recap_file) 
        recap_writer.writerow(['name','type','size']) #scrittura della prima row per ogni sessione del file csv, questa cosa non era richiesta ma ho voluto aggiungerla io per comodità e leggibilità del file excel
        for items in directory:     
            file_infos = os.stat(items) #metodo secondo il quale si ottengono i dati riguardanti il file preso in considerazione
            file = items.split('.') #metodo che divide i nomi dei file con un estensione contrassegnata da "." (utile per controllare e non sottoporre a controlli le cartelle)
            try: #nei if seguenti si controlla l'estensione del file e lo si inserisce nella cartella corretta (non ho aggiunto ogni singola estensione esistente vista la quantità innumerevole di esse), mi sono limitato ad aggiungere quelle necessarie allo svoglimento dell'esercizio e qualcuna in più come esempio
                if file[1] == "mp3" or file[1] == "wav" or file[1] == "mp1" or file[1] == "mp2" or file[1] == "mid" or file[1] == "midi":
                        shutil.move(items, "audio") #il metodo move delal librerira shutil ci permette di spostare i file nei percorsi dati (in questo caso ho usato dei percorsi relativi)
                        recap_writer.writerow([file[0],'audio',file_infos.st_size])
                if file[1] == "odt" or file[1] == "docx" or file[1] == "txt":
                        shutil.move(items, "documents")
                        recap_writer.writerow([file[0],'document',file_infos.st_size])
                if file[1] == "png" or file[1] =="jpeg" or file[1] == "jpg" or file[1] == "jfif":
                        shutil.move(items, "img")
                        recap_writer.writerow([file[0],'image',file_infos.st_size])
            except:
                print("That's a directory or the file is not recognizable") #eccezzione d'errore nel metodo try che viene castata nel momento in cui 
def imgdescriptioner():
    os.chdir("C:\\Users\\manci\\Desktop\\FileOrganizer\\files\\img") #percorso assoluto che ci porta nella cartella img
    table = [["name","height","width","grayscale","R","G","B","ALPHA"]] #parte iniziale della tabella 
    for image in os.listdir(): #iterazione di ogni file nella cartella img
        img = Image.open(image) #metodo della libreria PIL (pillow) che ci permette di aprire l'immagine ed esaminarla
        imgarr = np.array(img) #metodo della libreria numpy che ci permette di creare un array con i valori dei pixel dell'immagine
        sizes = imgarr.shape #ottengo una tupla con i valori (altezza,larghezza e nel caso in cui siano a colori il numero di layer 3 nel caso dell'RGB e 4 nel caso dell'RGBA)
        
        try: #i controlli seguenti verificano se l'immagine sia RGB RGBA o in bianco e nero
            if sizes[2] == 3: #RGB
                r = imgarr[:,:,0] #primo strato di R
                g = imgarr[:,:,1] #secondo strato di G
                b = imgarr[:,:,2] #terzo strato di B
                table.append([image.split('.')[0],sizes[0],sizes[1],0 ,np.mean(r),np.mean(g),np.mean(b),"not RGBA"])
            elif sizes[2] == 4: #RGBA
                r = imgarr[:,:,0]
                g = imgarr[:,:,1]
                b = imgarr[:,:,2]
                a = imgarr[:,:,3] #quarto strato di ALPHA
                table.append([image.split('.')[0],sizes[0],sizes[1],0 ,np.mean(r),np.mean(g),np.mean(b),np.mean(a)])
        except: # nel caso in cui il valore dei layer non sia presente nella tupla quindi si generi un errore l'immagine è in bianco e nero 
            table.append([image.split('.')[0],sizes[0],sizes[1],np.mean(imgarr),0,0,0,"not RGBA"])
    print(tabulate(table)) 
    os.chdir("..") # ".." ci permette di tornare nella cartella files
    

dirchecker(os.listdir()) #invochiamo il metodo e diamo come argmomento la lista dei files della directory con il metodo listdir()
dirmover(os.listdir()) 
imgdescriptioner() 

That's a directory or the file is not recognizable
That's a directory or the file is not recognizable
That's a directory or the file is not recognizable
--------  ------  -----  ------------------  ------------------  ------------------  ------------------  -----------------
name      height  width  grayscale           R                   G                   B                   ALPHA
bw        512     512    21.480064392089844  0                   0                   0                   not RGBA
daffodil  500     335    0                   109.25114626865671  85.55517611940299   4.967289552238806   not RGBA
download  168     300    0                   93.91027777777778   96.84200396825396   101.88075396825397  not RGBA
eclipse   256     256    0                   109.04953002929688  109.52186584472656  39.8489990234375    133.5909423828125
trump     183     275    0                   97.01080973671138   98.99407848981619   90.9237953303527    not RGBA
--------  ------  -----  ---------